In [54]:
import sys
print(sys.path)
sys.path.insert(2, '/home/odyssey/mmk_smoke_detection/')
print(sys.path)

['/home/odyssey/mmk_smoke_detection/validation', '/usr/lib/python38.zip', '/home/odyssey/mmk_smoke_detection/', '/home/odyssey/mmk_smoke_detection/', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/odyssey/.local/lib/python3.8/site-packages', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/home/odyssey/.local/lib/python3.8/site-packages/IPython/extensions', '/home/odyssey/.ipython']
['/home/odyssey/mmk_smoke_detection/validation', '/usr/lib/python38.zip', '/home/odyssey/mmk_smoke_detection/', '/home/odyssey/mmk_smoke_detection/', '/home/odyssey/mmk_smoke_detection/', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/odyssey/.local/lib/python3.8/site-packages', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/home/odyssey/.local/lib/python3.8/site-packages/IPython/extensions', '/home/odyssey/.ipython']


In [55]:
from dataset_preparator.preparator import plot_detections
from dataset_preparator.classJson import JsonData
PATH_TO_CONF = "/home/odyssey/mmk_smoke_detection/validation/new_conf"

In [56]:
from typing import Dict
import glob
import os

class ConfigController:
    configs: Dict[str, JsonData]
        
    @staticmethod
    def extract_key_from_conf_filename(
        conf_filename: str
    ) -> str:
        return os.path.basename(conf_filename)[5:13]
    
    def __init__(
        self,
        path_to_configs: str,
    ):
        self.configs = {}
        conf_files = glob.glob(os.path.join(path_to_configs, '*'))
        print(conf_files)
        for conf_file in conf_files:
            conf_key = self.extract_key_from_conf_filename(conf_file)
            conf_data = JsonData(conf_file)
            self.configs[conf_key] = conf_data
            
    def __getitem__(
        self,
        key: str
    ) -> JsonData:
        return self.configs[key]
        

In [57]:
conf_con = ConfigController(PATH_TO_CONF)
# list(conf_con.configs.keys())
conf_con['ct131405']

['/home/odyssey/mmk_smoke_detection/validation/new_conf/conf_ct131406.json', '/home/odyssey/mmk_smoke_detection/validation/new_conf/conf_ct131405.json', '/home/odyssey/mmk_smoke_detection/validation/new_conf/conf_ct131402.json', '/home/odyssey/mmk_smoke_detection/validation/new_conf/conf_ct131401.json', '/home/odyssey/mmk_smoke_detection/validation/new_conf/conf_ct131403.json', '/home/odyssey/mmk_smoke_detection/validation/new_conf/conf_ct131404.json']


In [58]:
!ls 05.10.21 

ct131401  ct131402  ct131403  ct131404	ct131405  ct131406  ct1314.csv


In [59]:
LABELS = ["background", "emission", "fire", "machine"]
CATEGORY_INDEX = {
    label_idx: {"id": label_idx, "name": label}
    for label_idx, label in enumerate(LABELS)
}

In [60]:
PATH_TO_VALIDATION_DATA = "05.10.21"

validation_camera_keys = [
    path.split('/')[1]
    for path in glob.glob(os.path.join(PATH_TO_VALIDATION_DATA, '*'))
    if os.path.isdir(path)
]

In [61]:
validation_camera_keys

['ct131402', 'ct131406', 'ct131405', 'ct131404', 'ct131401', 'ct131403']

In [117]:
import random

random_image_for_key = random.choice(
    glob.glob(
        os.path.join(PATH_TO_VALIDATION_DATA, validation_camera_keys[0], "emission", "**", "*")
    )
)

In [118]:
random_image_for_key

'05.10.21/ct131402/emission/weak/60a5f771-2a5e-493a-87c3-16da9405ccbd.jpg'

In [119]:
import cv2
import numpy as np
from PIL import Image, ImageDraw

frame = np.array(Image.open(random_image_for_key))
pers_matrix = conf_con[validation_camera_keys[0]].return_matrix()
# frame = cv2.warpPerspective(
#     np.array(frame),
#     np.array(pers_matrix['matrix']),
#     (pers_matrix['maxWidth'], pers_matrix['maxHeight']),
#     flags=cv2.INTER_LINEAR
# )

In [120]:
(pers_matrix['maxWidth'], pers_matrix['maxHeight'])

(1391, 940)

In [121]:
frame.shape

(1080, 1920, 3)

In [122]:
Image.fromarray(frame).show()

In [123]:
import pandas as pd
PATH_TO_CSV = "05.10.21/ct1314.csv"
table = pd.read_csv(PATH_TO_CSV)
table.head()

,paths,x_min,x_max,y_min,y_max,class
0,ct131401/background/900bae5e-94e7-42db-82c1-2d...,287,341,877,919,background
1,ct131401/background/ae85e990-96f2-4c39-8e09-5a...,408,454,609,649,background
2,ct131401/background/6692a25b-1fde-4f26-ace3-d3...,474,515,457,494,background
3,ct131401/background/b95bf74a-6c02-454f-922f-76...,489,530,408,439,background
4,ct131401/background/5c866374-7b9e-47a4-9d3e-3a...,321,372,799,845,background


In [124]:
os.path.join(*random_image_for_key.split('/')[1:])

'ct131402/emission/weak/60a5f771-2a5e-493a-87c3-16da9405ccbd.jpg'

In [125]:
table_path = os.path.join(*random_image_for_key.split('/')[1:])
path_table_part = table[table['paths'] == table_path]

In [126]:
path_table_part

,paths,x_min,x_max,y_min,y_max,class
180,ct131402/emission/weak/60a5f771-2a5e-493a-87c3...,517,576,898,942,emission


In [127]:
path_table_part['class'].to_list()[0]

'emission'

In [260]:
boxes = path_table_part[['y_min', 'x_min', 'y_max', 'x_max']].to_numpy()
classes = np.array([LABELS.index(path_table_part['class'].to_list()[0])])
scores = np.array([1.0])

plot_detections(
    image_np=frame,
    boxes=np.array(boxes),
    classes=np.array(classes),
    scores=np.array(scores),
    category_index=CATEGORY_INDEX,
    image_name="test_draw.jpg",
    line_thickness=[2],
    use_normalized_coordinates=False
)

In [257]:
# test_boxes = path_table_part[['y_min', 'x_min', 'y_max', 'x_max']].to_numpy()
test_boxes = path_table_part[['x_min', 'y_min', 'x_max', 'y_max']].to_numpy()
test_boxes = np.float32(test_boxes).reshape(-1,1,2)
mat = np.array(pers_matrix['matrix'])
pers_box = cv2.perspectiveTransform(test_boxes, mat).reshape(4).astype(int)
pers_box = [[pers_box[1], pers_box[0], pers_box[3], pers_box[2]]]

(2, 1, 2)


In [259]:
pers_frame = cv2.warpPerspective(
    np.array(frame),
    np.array(pers_matrix['matrix']),
    (pers_matrix['maxWidth'], pers_matrix['maxHeight']),
    flags=cv2.INTER_LINEAR
)

print(pers_frame.shape)

plot_detections(
    image_np=pers_frame,
    boxes=np.array(pers_box),
    classes=np.array(classes),
    scores=np.array(scores),
    category_index=CATEGORY_INDEX,
    image_name="box_test_draw.jpg",
    line_thickness=[6],
    use_normalized_coordinates=False
)


(940, 1391, 3)


In [250]:
Image.fromarray(pers_frame).save('test_pers.jpg')